<a href="https://colab.research.google.com/github/saahiljain/google_colab/blob/master/getting_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# mounting google drive to colab to get the training data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd /content/drive/My\ Drive

/content/drive/My Drive


In [0]:
!wget https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv

--2019-04-22 10:34:56--  https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c09::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1194033454 (1.1G) [text/csv]
Saving to: ‘train-annotations-bbox.csv.1’

train-annotations-b 100%[===================>]   1.11G  28.4MB/s    in 31s     

2019-04-22 10:35:27 (37.0 MB/s) - ‘train-annotations-bbox.csv.1’ saved [1194033454/1194033454]



In [0]:
!wget https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv

--2019-04-22 10:35:30--  https://storage.googleapis.com/openimages/2018_04/class-descriptions-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11255 (11K) [text/csv]
Saving to: ‘class-descriptions-boxable.csv’

class-descriptions- 100%[===================>]  10.99K  --.-KB/s    in 0.003s  

2019-04-22 10:35:30 (3.98 MB/s) - ‘class-descriptions-boxable.csv’ saved [11255/11255]



In [0]:
!pip3 install awscli

In [0]:
!mkdir JPEGImages

In [0]:
!mkdir labels

In [0]:
import csv
import subprocess
import os

runMode = "train"
classes = ["Ambulance"]#change here to get ambulance

with open('class-descriptions-boxable.csv', mode='r') as infile:
    reader = csv.reader(infile)
    dict_list = {rows[1]:rows[0] for rows in reader}

subprocess.run([ '!mkdir', 'JPEGImages'],shell=True)

subprocess.run(['rm', '-rf', 'labels'],shell=True)
subprocess.run([ '!mkdir', 'labels'],shell=True)

for ind in range(0, len(classes)):
    
    className = classes[ind]
    print("Class " + str(ind) + " : " + className)

    commandStr = "grep " + dict_list[className] + " " + runMode + "-annotations-bbox.csv"
    print(commandStr)
    class_annotations = subprocess.run(commandStr.split(), stdout=subprocess.PIPE).stdout.decode('utf-8')
    class_annotations = class_annotations.splitlines()

    totalNumOfAnnotations = len(class_annotations)
    print("Total number of annotations : "+str(totalNumOfAnnotations))

    cnt = 0
    for line in class_annotations[0:totalNumOfAnnotations]:
        cnt = cnt + 1
        print("annotation count : val " + str(cnt))
        lineParts = line.split(',')
        print(lineParts)
        subprocess.run([ 'aws', 's3', '--no-sign-request', '--only-show-errors', 'cp', 's3://open-images-dataset/'+runMode+'/'+lineParts[0]+".jpg", 'JPEGImages/'+lineParts[0]+".jpg"])
        with open('labels/%s.txt'%(lineParts[0]),'a') as f:
            f.write(' '.join([str(ind),str((float(lineParts[5]) + float(lineParts[4]))/2), str((float(lineParts[7]) + float(lineParts[6]))/2), str(float(lineParts[5])-float(lineParts[4])),str(float(lineParts[7])-float(lineParts[6]))])+'\n')

print("done")


Class 0 : Ambulance
grep /m/012n7d train-annotations-bbox.csv
Total number of annotations : 160
annotation count : val 1
['002cff1919a39fe7', 'xclick', '/m/012n7d', '1', '0.050625', '0.965000', '0.228333', '0.833333', '1', '0', '0', '0', '0']
annotation count : val 2
['00510a8d1b746d21', 'xclick', '/m/012n7d', '1', '0.000000', '0.838608', '0.040378', '0.806701', '1', '1', '0', '0', '0']


KeyboardInterrupt: ignored